In [93]:
%load_ext autoreload
%autoreload 2

import cv2
import matplotlib.pyplot as plt
import numpy as np
import os
import rasterio
import seaborn as sns
import torch

from evolver import MatrixEvolver
from unet import UNet

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


IndentationError: unexpected indent (evolver.py, line 20)

In [58]:
def read_tif_to_np(tif_path):
    """Reads a tif file and converts it into a numpy.ndarray.
    
    Arg:
        tif_path: The full path to the tif file to read.
    
    Returns:
        A numpy.ndarray containing the tif file data.
    
    """
    with rasterio.open(tif_path) as f:
        data = np.rollaxis(f.read(), 0, 3)
        return data
    
def get_tile_y(base_dir, tile):
    """Retrieves the landcover y data for a given tile.
    
    Args:
        base_dir: A directory to a tile block.
        tile: A tile name of format {block}/{block}{quad}_{quarter}_{garbage}.
    
    Returns:
        A ndarray containing the y data for the tile.
    
    """
    y_dir_ext = "_lc.tif"
    y_tile_path = base_dir + tile + y_dir_ext
    return read_tif_to_np(y_tile_path)

def get_tile_x(base_dir, tile):
    """Retrieves the landcover x data for a given tile.
    
    Args:
        base_dir: A directory to a tile block.
        tile: A tile name of format {block}/{block}{quad}_{quarter}_{garbage}.
    
    Returns:
        A ndarray) containing the x data for the tile.
    
    """
    x_dir_ext = ".tif" 
    x_tile_path = base_dir + tile + x_dir_ext
    return read_tif_to_np(x_tile_path)

def apply_label_remap(labels, label_map):
    """Reassigns labels inplace in an numpy array given a provided mapping.
    
    Args:
        labels: An ndarray of labels.
        label_map: A dict[int, int] mapping label classes [original, new].
        
    """
    for l1, l2 in label_map.items():
        labels[labels == l1] = l2

In [59]:
# Define tiles to be used in testing, training and finetuning.
tile_dir_y = "/mnt/blobfuse/resampled-lc/data/v1/2015/states/md/md_1m_2015/39076/"
tile_dir_x = "/mnt/blobfuse/esri-naip/v002/md/2015/md_100cm_2015/39076/" 
tiles = ["m_3907639_sw_18_1_20150815", "m_3907639_ne_18_1_20150815"]

In [60]:
# Read X and Y data for the tile and apply label transformations.
tile = tiles[0]
x = get_tile_x(tile_dir_x, tile)
y = get_tile_y(tile_dir_y, tile)

water_forest_land_impervious_remap = {5: 4, 6: 4}
apply_label_remap(y, water_forest_land_impervious_remap)

In [92]:
model = UNet(in_channels = 256, n_classes = len(np.unique(y)), depth = 4)
dropout_mask_evolver = 